In [116]:
import chess
import re
import numpy as np

game = '''
[Event "Rated Bullet tournament https://lichess.org/tournament/yc1WW2Ox"]
[Site "https://lichess.org/PpwPOZMq"]
[Date "2017.04.01"]
[Round "-"]
[White "Abbot"]
[Black "Costello"]
[Result "0-1"]
[UTCDate "2017.04.01"]
[UTCTime "11:32:01"]
[WhiteElo "2100"]
[BlackElo "2000"]
[WhiteRatingDiff "-4"]
[BlackRatingDiff "+1"]
[WhiteTitle "FM"]
[ECO "B30"]
[Opening "Sicilian Defense: Old Sicilian"]
[TimeControl "300+0"]
[Termination "Time forfeit"]

1. e4 { [%eval 0.17] [%clk 0:00:30] } 1... c5 { [%eval 0.19] [%clk 0:00:30] }
2. Nf3 { [%eval 0.25] [%clk 0:00:29] } 2... Nc6 { [%eval 0.33] [%clk 0:00:30] }
3. Bc4 { [%eval -0.13] [%clk 0:00:28] } 3... e6 { [%eval -0.04] [%clk 0:00:30] }
4. c3 { [%eval -0.4] [%clk 0:00:27] } 4... b5? { [%eval 1.18] [%clk 0:00:30] }
5. Bb3?! { [%eval 0.21] [%clk 0:00:26] } 5... c4 { [%eval 0.32] [%clk 0:00:29] }
6. Bc2 { [%eval 0.2] [%clk 0:00:25] } 6... a5 { [%eval 0.6] [%clk 0:00:29] }
7. d4 { [%eval 0.29] [%clk 0:00:23] } 7... cxd3 { [%eval 0.6] [%clk 0:00:27] }
8. Qxd3 { [%eval 0.12] [%clk 0:00:22] } 8... Nf6 { [%eval 0.52] [%clk 0:00:26] }
9. e5 { [%eval 0.39] [%clk 0:00:21] } 9... Nd5 { [%eval 0.45] [%clk 0:00:25] }
10. Bg5?! { [%eval -0.44] [%clk 0:00:18] } 10... Qc7 { [%eval -0.12] [%clk 0:00:23] }
11. Nbd2?? { [%eval -3.15] [%clk 0:00:14] } 11... h6 { [%eval -2.99] [%clk 0:00:23] }
12. Bh4 { [%eval -3.0] [%clk 0:00:11] } 12... Ba6? { [%eval -0.12] [%clk 0:00:23] }
13. b3?? { [%eval -4.14] [%clk 0:00:02] } 13... Nf4? { [%eval -2.73] [%clk 0:00:21] } 0-1
'''

In [95]:
def get_ratings(game_string):

    white_rating = int(re.search(r'\[WhiteElo "(\d+)"\]', game).group(1))
    black_rating = int(re.search(r'\[BlackElo "(\d+)"\]', game).group(1))

    return white_rating, black_rating


White rating: 2100
Black rating: 2000


In [127]:

def get_game_tensors(game_string,num_tensors):

    gt = np.zeros((num_tensors,134))

    moves = []
    evals = []

    for move in re.findall(r'\d+\.+\s+(\D+\d)[\?|\!]*\s\{\s\[%eval (\#?\-?\d+\.?\d*)',game_string):
        moves.append(move[0])
        evals.append(move[1])

    #let our t vector be a 1D array of 133 elements. The first 128 element represent the board before the move is made and after the move is made. The 129th element is the evaluation of the move before it is made and the 130th element is the evaluation of the move after it is made. If it is mate in X moves before the move is made, the 131st element is 1 and 0 otherwise. If it is mate in X moves after the move is made, the 132nd element is 1 and 0 otherwise. The 133rd element is 1 if it is white moved and -1 if it is black making the move.

    board = chess.Board()

    for m in range(0,min(40,len(moves)-1)):
        t = np.zeros(134)

        for i in range(64): #original board position
            if board.piece_at(i) is not None:
                t[i] = board.piece_at(i).piece_type * (1 if board.piece_at(i).color == chess.WHITE else -1)
    
        board.push_san(moves[m])
        for i in range(64):
            if board.piece_at(i) is not None:
                t[i+64] = board.piece_at(i).piece_type * (1 if board.piece_at(i).color == chess.WHITE else -1)
    
        #evals is either a number, or starts with a #. If it starts with a #, it is a mate in X moves
        if evals[m].startswith('#'):
            t[129] = float(evals[m][1:])
            t[130] = 1
        else:
            t[129] = float(evals[m])
            t[130] = 0
    
        if evals[m+1].startswith('#'):
            t[131] = float(evals[m+1][1:])
            t[132] = 1
        else:
            t[131] = float(evals[m+1])
            t[132] = 0

        t[133] = -1 if board.turn == chess.WHITE else 1

        gt[m] = t

    return gt

In [128]:
get_game_tensors(game,40)

array([[ 4.  ,  2.  ,  3.  , ...,  0.19,  0.  ,  1.  ],
       [ 4.  ,  2.  ,  3.  , ...,  0.25,  0.  , -1.  ],
       [ 4.  ,  2.  ,  3.  , ...,  0.33,  0.  ,  1.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]], shape=(40, 134))